In [49]:
import pickle

In [ ]:

import json
import pickle
from o2.models.settings import Settings
from o2.store import Store


Settings.LOG_FILE = "IO"

#file = "o2_evaluation/redumped_stores/store_proximal_policy_optimization_production_mid.pkl"
file = "o2_evaluation/redumped_stores/store_simulated_annealing_random_bpi_challenge_2012_mid.pkl"
with open(file, "rb") as f:
    store:Store = pickle.load(f)

solution = store.solution

print(json.dumps(solution.state.timetable.to_dict()["batch_processing"], indent=4))


In [ ]:
from prosimos.simulation_engine import run_simpy_simulation

from o2.models.evaluation import Evaluation
from o2.simulation_runner import SimulationRunner

solution = pareto.solutions[0]
print(solution.point)
print(solution.id)
print(solution.evaluation)
print(solution.evaluation.get_avg_processing_cost_per_task())

print(state.timetable.batch_processing)

state = solution.state

low = None
high = None
while True:
    setup = state.to_sim_diff_setup()
    result = SimulationRunner.run_simulation(state)
    evaluation = Evaluation.from_run_simulation_result(
            state.timetable.get_hourly_rates(),
            state.timetable.get_fixed_cost_fns(),
            state.timetable.batching_rules_exist,
            result,
        )
    
    # if evaluation.pareto_x < 2000:
    #     low = result
    # if evaluation.pareto_x > 200_000:
    #     high = result
    # if low is not None and high is not None:
    #     break
    if evaluation.is_empty:
        low = result 
    else:
        high = result

    if low is not None and high is not None:
        break





In [20]:
low_evaluation = Evaluation.from_run_simulation_result(
            state.timetable.get_hourly_rates(),
            state.timetable.get_fixed_cost_fns(),
            state.timetable.batching_rules_exist,
            low,
        )

In [ ]:
low_evaluation.avg_cost_by_case

In [ ]:



assert low is not None
assert high is not None
_,_, _, log_info_low = low
_,_, _, log_info_high = high
print(
    "\n".join(
        [
            f"{event.p_case} {event.task_id}:\t{event.started_datetime.isoformat()} -> {event.completed_datetime.isoformat()} (enabled: {event.enabled_datetime.isoformat()}) (I:{event.idle_time / 3600}, P:{event.processing_time/3600}, WT:{event.waiting_time/3600}, C:{event.cycle_time / 3600})"
            for trace in log_info_low.trace_list
            for event in trace.event_list
        ]
    )
)
print("---------------------")
print(
    "\n".join(
        [
            f"{event.p_case} {event.task_id}:\t{event.started_datetime.isoformat()} -> {event.completed_datetime.isoformat()} (enabled: {event.enabled_datetime.isoformat()}) (I:{event.idle_time / 3600}, P:{event.processing_time/3600}, WT:{event.waiting_time/3600}, C:{event.cycle_time / 3600})"
            for trace in log_info_high.trace_list
            for event in trace.event_list
        ]
    )
)


In [ ]:
e_high = Evaluation.from_run_simulation_result(
            state.timetable.get_hourly_rates(),
            state.timetable.get_fixed_cost_fns(),
            state.timetable.batching_rules_exist,
            high,
        )

print(e_high.is_empty)


In [1]:
coords1 = [(81767474.20132078, 20508784.314677723), (84169591.79665251, 1.003725801310618), (44004614.57269864, 22295763.81033023), (84883015.33665407, 1.0007589610468888), (83237160.55003728, 12996396.74595918), (84225092.67340529, 1.001768337364194), (83756781.0176239, 10296517.047934439), (81967963.13062094, 14455610.3138614), (86455352.50196101, 1.0007117428024608), (82630989.60231191, 14227796.126775883), (87999344.71894236, 1.0007109256878903), (85204638.7676439, 1.0007242359240853)]
coords2 = [(86347972.82794288, 1.0067388136448336), (81967963.13062094, 14455610.3138614), (84101598.7982898, 12603031.1645852), (84225092.67340529, 1.001768337364194), (84169591.79665251, 1.003725801310618), (84883015.33665407, 1.0007589610468888), (84002955.70935139, 12821594.602079961), (83237160.55003728, 12996396.74595918), (81767474.20132078, 20508784.314677723), (82630989.60231191, 14227796.126775883)]
ref_point = (25675785168.695038, 29513005161.43333)

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import box
from shapely.ops import unary_union

# Your data:
pareto1 = [
    (81767474.20132078, 20508784.314677723),
    (84169591.79665251, 1.003725801310618),
    (44004614.57269864, 22295763.81033023),
    (84883015.33665407, 1.0007589610468888),
    (83237160.55003728, 12996396.74595918),
    (84225092.67340529, 1.001768337364194),
    (83756781.0176239, 10296517.047934439),
    (81967963.13062094, 14455610.3138614),
    (86455352.50196101, 1.0007117428024608),
    (82630989.60231191, 14227796.126775883),
    (87999344.71894236, 1.0007109256878903),
    (85204638.7676439, 1.0007242359240853)
]
pareto2 = [
    (86347972.82794288, 1.0067388136448336),
    (81967963.13062094, 14455610.3138614),
    (84101598.7982898, 12603031.1645852),
    (84225092.67340529, 1.001768337364194),
    (84169591.79665251, 1.003725801310618),
    (84883015.33665407, 1.0007589610468888),
    (84002955.70935139, 12821594.602079961),
    (83237160.55003728, 12996396.74595918),
    (81767474.20132078, 20508784.314677723),
    (82630989.60231191, 14227796.126775883)
]
ref_point = (25675785168.695038, 29513005161.43333)

# Function to build the hypervolume region from a Pareto front:
def hypervolume_polygon(pareto_points, ref):
    # Each Pareto point gives a box from the point to the reference point.
    boxes = [box(pt[0], pt[1], ref[0], ref[1]) for pt in pareto_points]
    # The union of all these boxes gives the overall dominated area (the hypervolume).
    hv_poly = unary_union(boxes)
    return hv_poly

# Compute hypervolume polygons for each front:
hv1 = hypervolume_polygon(pareto1, ref_point)
hv2 = hypervolume_polygon(pareto2, ref_point)

# Compute the intersection and the differences:
intersection = hv1.intersection(hv2)
hv1_only = hv1.difference(intersection)
hv2_only = hv2.difference(intersection)

# Plotting:
fig, ax = plt.subplots(figsize=(8, 6))

def plot_polygon(poly, color, label):
    if poly.is_empty:
        return
    if poly.geom_type == 'Polygon':
        xs, ys = poly.exterior.xy
        ax.fill(xs, ys, color=color, alpha=0.5, label=label)
    elif poly.geom_type == 'MultiPolygon':
        for p in poly.geoms:
            xs, ys = p.exterior.xy
            ax.fill(xs, ys, color=color, alpha=0.5, label=label)


# Plot unique regions and intersection:
plot_polygon(hv1_only, 'red', 'Pareto1 Only')
plot_polygon(hv2_only, 'blue', 'Pareto2 Only')
plot_polygon(intersection, 'green', 'Intersection')

ax.set_xlabel('Objective 1')
ax.set_ylabel('Objective 2')
ax.legend()
plt.title('Hypervolume Regions for Two Pareto Fronts')
plt.show()


In [1]:
import pickle

s = pickle.load(open("stores/run_2025-03-03_20-15-21/store_simulated_annealing_production_easy.pkl", "rb"))

min_cycle_time_solution = min(s.pareto_fronts[-1].solutions, key=lambda x: x.evaluation.avg_cycle_time_by_case)


In [ ]:
import json
from dataclasses import replace
from o2.models.timetable import (
    BATCH_TYPE,
    RULE_TYPE,
    BatchingRule,
    Distribution,
    FiringRule,
    RuleSelector,
)
from o2.models.evaluation import Evaluation
from o2.simulation_runner import SimulationRunner


task_id = "node_8e888ff4-54dd-4a03-9da2-9a82ae49067e"


new_eval = min_cycle_time_solution.state.evaluate()
print(new_eval.avg_processing_time_per_batch)
print(new_eval.avg_cycle_time_by_case)
print(new_eval.task_kpis[task_id].processing_time.total / new_eval.task_kpis[task_id].processing_time.count)

#print(min_cycle_time_solution.evaluation.avg_batch_size_per_task[task_id])

sol = min_cycle_time_solution
timetable = sol.state.timetable

# new_timetable= timetable.replace_firing_rule(
#     RuleSelector(
#         batching_rule_task_id="node_8e888ff4-54dd-4a03-9da2-9a82ae49067e",
#         firing_rule_index=(0, 3),
#     ),
#     FiringRule.gte(
#         RULE_TYPE.SIZE,
#         5,
#     )
# )

#fn = s.constraints.get_duration_fn_for_task(task_id)

fn = "1/(4*size)"

# new_timetable= timetable.add_firing_rule(
#     RuleSelector(
#         batching_rule_task_id=task_id,
#         firing_rule_index=None,
#     ),
#     FiringRule.gte(RULE_TYPE.SIZE, 10),
#     duration_fn=fn,
# )

new_timetable = timetable.add_firing_rule(
    RuleSelector(
        batching_rule_task_id=task_id,
        firing_rule_index=None,
    ),
    FiringRule.gte(RULE_TYPE.SIZE, 20),
    duration_fn=fn,
)

new_state = replace(
    sol.state,
    timetable=new_timetable,
)

new_evaluation = new_state.evaluate()
print(fn)
print(new_evaluation.avg_processing_time_per_batch)
print(new_evaluation.avg_cycle_time_by_case)
print(new_evaluation.task_kpis[task_id].processing_time.total / new_evaluation.task_kpis[task_id].processing_time.count)
print(new_evaluation.avg_batch_size_per_task[task_id])

result = SimulationRunner.run_simulation(new_state)
evaluation = Evaluation.from_run_simulation_result(
        new_state.timetable.get_hourly_rates(),
        new_state.timetable.get_fixed_cost_fns(),
        new_state.timetable.batching_rules_exist,
        result,
    )

_,_, _, log_info = result
# print(
#     "\n".join(
#         [
#             f"{event.p_case} {event.task_id}:\t{event.started_datetime.isoformat()} -> {event.completed_datetime.isoformat()} (enabled: {event.enabled_datetime.isoformat()}) (I:{event.idle_time / 3600}, P:{event.processing_time/3600}, WT:{event.waiting_time/3600}, C:{event.cycle_time / 3600})"
#             for trace in log_info.trace_list
#             for event in trace.event_list
#         ]
#     )
# )


# # Pretty print json timetable
# print(json.dumps(new_timetable.to_dict()["batch_processing"], indent=4))


In [ ]:
new_evaluation.task_kpis[task_id].processing_time.avg

In [34]:
setup = new_state.to_sim_diff_setup()


In [ ]:
setup.batch_processing['node_8e888ff4-54dd-4a03-9da2-9a82ae49067e'].duration_distribution

In [ ]:
results = []
results_count = []
for i in range(100):
    print(i)
    new_eval = new_state.evaluate()
    results.append(new_eval.avg_processing_time_per_batch)
    results_count.append(new_eval.task_kpis[task_id].processing_time.count)

# Calculate the average of the results and then calculate the average error
average = sum(results) / len(results)

mean_average_error = sum(abs(result - average) for result in results) / len(results)
print(mean_average_error)
# In Percentage
print(mean_average_error / average)

average_count = sum(results_count) / len(results_count)
print(average_count)

average_count_error = sum(abs(result - average_count) for result in results_count) / len(results_count)
print(average_count_error)

average_count_error_percentage = average_count_error / average_count
print(average_count_error_percentage)


In [ ]:
# Generate 10_000 random points in 0:100
import numpy as np
import matplotlib.pyplot as plt

points = np.random.rand(100_000, 2) * 100

center_point = (45, 65)
error_percentage = 0.05

error_radius = error_percentage * np.sqrt(center_point[0]**2 + center_point[1]**2)
#error_radius = 55 * error_percentage
print (error_radius)

# Draw a scatter plot of the points, color the points within the error radius red
plt.scatter(points[:, 0], points[:, 1], c=np.where(np.linalg.norm(points - center_point, axis=1) <= error_radius, 'red', 'blue'), s=0.1)
plt.show()


In [34]:
from copy import deepcopy

from o2.util.helper import hash_int


def make_hash(o):

  """
  Makes a hash from a dictionary, list, tuple or set to any level, that contains
  only other hashable types (including any lists, tuples, sets, and
  dictionaries).
  """

  if isinstance(o, (set, tuple, list)):

    return tuple([make_hash(e) for e in o])    

  elif not isinstance(o, dict):

    return hash_int(o)

  new_o = deepcopy(o)
  for k, v in new_o.items():
    new_o[k] = make_hash(v)

  return hash_int(tuple(frozenset(sorted(new_o.items()))))

In [35]:
from dataclasses import fields, is_dataclass
from typing import Any

from pydantic import BaseModel

from o2.models.time_period import TimePeriod
from o2.util.helper import hash_int


def deep_hash(obj: Any) -> int:
    """ Recursively compute a hash for nested frozen dataclasses and other basic types. """
    if isinstance(obj, (str, int, float, bool, bytes, type(None))):
        return hash_int(obj)

    elif isinstance(obj, (list, tuple)):
        # Hash sequences by hashing the tuple of item hashes
        return hash_int(tuple(deep_hash(item) for item in obj))

    elif isinstance(obj, dict):
        # Hash dict by hashing sorted (key, value) pairs
        return hash_int(tuple(sorted((deep_hash(k), deep_hash(v)) for k, v in obj.items())))

    elif is_dataclass(obj):
        # Hash dataclass by hashing a tuple of its field hashes
        return hash_int(tuple(deep_hash(getattr(obj, f.name)) for f in fields(obj)))

    elif isinstance(obj, TimePeriod):
        return obj.id
    
    # support pydantic
    elif isinstance(obj, BaseModel):
        return hash_int(obj.model_dump_json())
    else:
        raise TypeError(f"Unsupported type: {type(obj)}")

In [ ]:
import pickle
state = pickle.load(open("stores/run_2025-03-04-23-10-00-422042/states/state_simulated_annealing_production_mid_1342702810.pkl", "rb"))


# Run 10_000 iterations for each hash technique. Start with hash(to_json). Time the execution.

import time

start_time = time.time()
for i in range(10_000):
    hash_int(state.timetable.to_json())
end_time = time.time()

avg_time = (end_time - start_time) / 10_000
total_time = end_time - start_time
print(f"Time taken: {avg_time:3f} (avg) {total_time:3f} (total) seconds")

# Now do the same for make_hash

# start_time = time.time()
# for i in range(10_000):
#     make_hash(state.timetable.to_dict())
# end_time = time.time()

# avg_time = (end_time - start_time) / 1_000
# total_time = end_time - start_time
# print(f"Time taken: {avg_time:3f} (avg) {total_time:3f} (total) seconds")

# Use deep_hash (to_dict)

start_time = time.time()
for i in range(10_000):
    deep_hash(state.timetable.to_dict())
end_time = time.time()

avg_time = (end_time - start_time) / 10_000
total_time = end_time - start_time
print(f"Time taken: {avg_time:3f} (avg) {total_time:3f} (total) seconds")

# Use deep_hash
start_time = time.time()
for i in range(10_000):
    deep_hash(state.timetable)
end_time = time.time()

avg_time = (end_time - start_time) / 10_000
total_time = end_time - start_time
print(f"Time taken: {avg_time:3f} (avg) {total_time:3f} (total) seconds")





In [ ]:
import pickle
state = pickle.load(open("stores/run_2025-03-04-23-10-00-422042/states/state_simulated_annealing_production_mid_1342702810.pkl", "rb"))

from dataclasses import fields, is_dataclass

fields(state.timetable.resource_calendars[0])

In [ ]:
import random
import xxhash


h = xxhash.xxh3_64_hexdigest(bytes(random.randint(0, 100000)))
print (h)
n = int(h, 16)
print (n)
hh = f"{n:x}"
print (hh)


In [ ]:
import rtree


tree = rtree.index.Index() 
tree.insert(29731438600781092, (0, 0, 0, 0))

get = next(tree.nearest((0, 0, 0, 0), 1, objects=True))
print (get.id)






In [1]:
import pickle
store = pickle.load(open("o2_evaluation/redumped_stores/store_tabu_search_bpi_challenge_2012_easy.pkl", "rb"))


In [ ]:
sols = [ (sol.evaluation.total_cycle_time) for sol in store.pareto_fronts[-1].solutions]

problem_solution = store.pareto_fronts[-1].solutions[sols.index(0)]
print(problem_solution.is_valid)
print(problem_solution.pareto_x, problem_solution.pareto_y)
print(problem_solution.evaluation.pareto_x, problem_solution.evaluation.pareto_y)
print(problem_solution.evaluation.is_empty)


In [ ]:


sol_id = problem_solution.id

evaluation_loaded = pickle.load(open(f"evaluations/evaluation_tabu_search_bpi_challenge_2012_easy_{sol_id}.pkl", "rb"))



In [ ]:
# Find with same id

for fronts in store.pareto_fronts:
    for sol in fronts.solutions:
        if sol.id == '230a931cf2a895ef':
            print(sol)
            break


Problem: Wir führen in einer Iteration zwei mal (A&B) die selbe Base-Aktion aus (aus verschiedenen Actions).
Dann wird A ausgeführt und angenommen. B wird nun auf dem neuen State S_A ausgeführt; B hat keinen Effekt mehr. Folglich wird B als invalide markiert, und so auch weg-gespeichert. Da A & B beide die selbe Action-Liste hatten, haben Sie auch die selbe id. 

Mögliche Lösung: Da A ja auf B angewendet wird, müsste die ID eigentlich aus ...->A->B bestehen, stat nur ...->A. 

Wobei. Wahrscheinlich tut sie das? Ich glaube das Anwenden auf die neue base solution gibts gar nicht mehr.

Das Grundkonzept könnte aber auch bei einem Fehler passieren: Wir probieren das selbe aus, einer der beiden schlägt fehl, überschreibt Ergebnis. 

In [ ]:
bad_result = None
bad_results_n = 0
for i in range(100):
    result = problem_solution.state.evaluate()
    if result.is_empty:
        bad_result = result
        bad_results_n += 1

print(bad_results_n )
print(bad_result)




In [ ]:
from o2.actions.batching_actions.modify_size_rule_by_allocation import (
    ModifySizeRuleByLowAllocationAction,
)
from o2.actions.batching_actions.modify_size_rule_by_cost_action import (
    ModifySizeRuleByCostAction,
)
from o2.agents.tabu_agent import TabuAgent
from o2.models.settings import Settings
from o2.models.solution import Solution
from o2.models.state import State
from o2.store import Store
from o2.util import logger
from o2.util.solution_dumper import SolutionDumper
from tests.fixtures.constraints_generator import ConstraintsGenerator
from tests.fixtures.store_fixture import ONE_TASK_BPMN_PATH
from tests.fixtures.timetable_generator import TimetableGenerator


class BrokenTabuAgent(TabuAgent):
    def __init__(self, store: Store) -> None:
        super().__init__(store)

        self.catalog = [
            ModifySizeRuleByLowAllocationAction,
            ModifySizeRuleByCostAction,
        ]

Settings.ARCHIVE_SOLUTIONS = True
Settings.LOG_LEVEL = "DEBUG"
logger.setup_logging()
SolutionDumper()
bpmn_content = open(ONE_TASK_BPMN_PATH).read()
state = State(
    bpmn_definition=bpmn_content,
    timetable=TimetableGenerator(bpmn_content).generate_simple(),
    for_testing=True,
)
store = Store.from_state_and_constraints(
    state, ConstraintsGenerator(state.bpmn_definition).add_size_constraint().generate()
)

agent = BrokenTabuAgent(store)

actions_to_perform = agent.select_actions(store)
assert actions_to_perform is not None
solution_tries = [
    Solution.from_parent(store.solution, action) for action in actions_to_perform
]
print(len(solution_tries))
good,bad = store.process_many_solutions(solution_tries)
for s in store.solution_tree.solution_lookup.values():
    if s is not None:
        s.archive()
Settings.ARCHIVE_SOLUTIONS = False
print(len(good))
print(len(bad))

In [ ]:
len(store.pareto_fronts[-1].solutions)

In [ ]:
import pickle
evaluation = pickle.load(open("/Users/jannis/Dropbox/Uni/Master/BP-Optimization/optimos_v2/stores/run_2025-03-05-22-29-48-205697/evaluations/evaluation__22ce0ac9860bc26a.pkl", "rb"))
evaluation.is_empty


In [20]:
store_name = "store_tabu_search_bpi_challenge_2012_hard.pkl"
store = pickle.load(open(f"o2_evaluation/redumped_stores/{store_name}", "rb"))


In [ ]:
print(store.pareto_fronts[-1].solutions[0].state.timetable.to_json())

In [ ]:
# Load every scenario .json file in the folder
import os
import glob

for file in glob.glob("o2_evaluation/redumped_stores/scenarios/*/*.json"):
    pass

In [ ]:
# Load State
import pickle
from o2.models.state import State
from o2.models.timetable import TimetableType
from o2.models.rule_selector import RuleSelector
from o2.models.timetable import BatchingRule, FiringRule, RULE_TYPE
from dataclasses import replace

solution_id = "0e08efe0422417b7"

# Load State
state: State = pickle.load(open(f"states/state_tabu_search_bpi_challenge_2012_hard_{solution_id}.pkl", "rb"))
timetable:TimetableType  = state.timetable

print(timetable.to_json())



selector1 =  RuleSelector("node_d45b421a-49fb-48c3-b8d9-fa77d9338e74", (4,0))
selector2 =  RuleSelector("node_1a561183-d10b-44f4-be71-e48ca4f32beb", (4,0))
new_timetable = timetable.replace_firing_rule(selector1, FiringRule.gte(RULE_TYPE.SIZE, 3)).replace_firing_rule(selector2, FiringRule.gte(RULE_TYPE.SIZE, 3))

new_state = replace(state, timetable=new_timetable)






In [ ]:
base = state.evaluate()


nom = 10
cycle_time = []
is_not_dominated = 0
for i in range(nom):
    new_eval = state.evaluate()
    cycle_time.append(new_eval.total_cycle_time)
    if not new_eval.is_dominated_by(base):
        is_not_dominated += 1

print(sum(cycle_time) / nom)
print(is_not_dominated / nom)

new_cycle_times = []
is_not_dominated = 0
for i in range(nom):
    new_eval = new_state.evaluate()
    new_cycle_times.append(new_eval.total_cycle_time)
    if not new_eval.is_dominated_by(base):
        is_not_dominated += 1

print(sum(new_cycle_times)/nom)
print(is_not_dominated / nom)



In [ ]:
from o2.agents.tabu_agent import TabuAgent
from o2.models.solution import Solution
new_base = new_state.evaluate()
store_name = "store_tabu_search_bpi_challenge_2012_hard.pkl"
store = pickle.load(open(f"o2_evaluation/redumped_stores/{store_name}", "rb"))

store.settings.only_allow_low_last = False
store.settings.max_number_of_actions_to_select = 999999

solution = Solution(actions=[], evaluation=new_base, state=new_state)
store.solution = solution
agent =  TabuAgent(store)
action_generators = [
    Action.rate_self(store, {}) for Action in agent.catalog # type: ignore
]

actions = TabuAgent.get_valid_actions(store, action_generators) # type: ignore

print(actions)






In [ ]:
from o2.actions.batching_actions.modify_size_rule_by_wt_action import ModifySizeRuleByWTAction


a1 = ModifySizeRuleByCostAction(params={'rule': RuleSelector(batching_rule_task_id='node_d45b421a-49fb-48c3-b8d9-fa77d9338e74', firing_rule_index=(4, 0)), 'size_increment': 1, 'duration_fn': '1 * size'})
a2 = ModifySizeRuleByWTAction(params={'rule': RuleSelector(batching_rule_task_id='node_d45b421a-49fb-48c3-b8d9-fa77d9338e74', firing_rule_index=(4, 0)), 'size_increment': -1, 'duration_fn': '1 * size'}))


In [ ]:
import pickle
store = pickle.load(open("/Users/jannis/Dropbox/Uni/Master/BP-Optimization/optimos_v2/stores/run_2025-03-08-20-51-09-270360/store_tabu_search_bpi_challenge_2012_mid.pkl", "rb"))

print(store.pareto_fronts[-1].solutions[0].state.timetable.to_json())

In [ ]:
# Comparison
import pickle
import json
from o2.store import Store
from o2.util.solution_dumper import SolutionDumper
from o2.models.settings import CostType, Settings
import os

Settings.COST_TYPE = CostType.AVG_WT_AND_PT_PER_TASK_INSTANCE
Settings.ARCHIVE_SOLUTIONS = True
Settings.DELETE_LOADED_SOLUTION_ARCHIVES = False
Settings.OVERWRITE_EXISTING_SOLUTION_ARCHIVES = False
Settings.CHECK_FOR_TIMETABLE_EQUALITY = True



os.chdir("/Users/jannis/Dropbox/Uni/Master/BP-Optimization/optimos_v2/")


SolutionDumper(global_mode=True)

storeNormal: Store = pickle.load(
    open(
        "/Users/jannis/Dropbox/Uni/Master/BP-Optimization/optimos_v2/o2_evaluation/redumped_stores/store_simulated_annealing_bpi_challenge_2012_mid.pkl",
        "rb",
    )
)
storeRandom: Store = pickle.load(
    open(
        "/Users/jannis/Dropbox/Uni/Master/BP-Optimization/optimos_v2/o2_evaluation/redumped_stores/store_simulated_annealing_random_bpi_challenge_2012_mid.pkl",
        "rb",
    )
)

for store in [storeNormal, storeRandom]:
    store_solutions = [
        *store.solution_tree.solution_lookup.values(),
        *[s for pareto in store.pareto_fronts for s in pareto.solutions],
    ]
    for solution in store_solutions:
        if solution is not None:
            # Add store name so we can load the evaluation and state later
            solution.__dict__["_store_name"] = store.name


minSoltutionNormal,minSoltutionNormalCT = sorted(
    [(s, s.evaluation.total_cycle_time) for s in storeNormal.current_pareto_front.solutions],
    key=lambda x: x[1],
    reverse=True,
)[0]

minSoltutionRandom,minSoltutionRandomCT = sorted(
    [(s, s.evaluation.total_cycle_time) for s in storeRandom.current_pareto_front.solutions],
    key=lambda x: x[1],
    reverse=True,
)[0]


print(f"minSoltutionNormalCT: {minSoltutionNormalCT:_.2f}, minSoltutionRandomCT: {minSoltutionRandomCT:_.2f}")


for normal_batching_rule in minSoltutionNormal.state.timetable.batch_processing:
    task_id = normal_batching_rule.task_id
    if len (minSoltutionRandom.state.timetable.get_batching_rules_for_task(task_id)) == 0:
        print(f"Rule for {task_id} not found in random")
        continue

    random_batching_rule = minSoltutionRandom.state.timetable.get_batching_rules_for_task(task_id)[0]

    if normal_batching_rule == random_batching_rule:
        print(f"Rule for {task_id} are equal, continuuing")
        continue

    # length = max(len(normal_batching_rule.firing_rules), len(random_batching_rule))

    # for i in range(length):
    #     print("Random":)

    print(f"\nRules for Task {task_id}")
    avg_batch_size_normal = minSoltutionNormal.evaluation.avg_batch_size_per_task.get(task_id,0)
    avg_batch_size_random = minSoltutionRandom.evaluation.avg_batch_size_per_task.get(task_id,0)
    print(f"Average Batch-Size -- Random: {avg_batch_size_random:_.1f}, Normal: {avg_batch_size_normal:_.1f}")
    
    print(f"Random:\n{json.dumps(random_batching_rule.to_dict()['firingRules'], indent=4)}")

    print(f"\n\nNormal:\n{json.dumps(normal_batching_rule.to_dict()['firingRules'], indent=4)}")


for random_batching_rule in minSoltutionRandom.state.timetable.batch_processing:
    task_id = random_batching_rule.task_id
    has_normal_br = len(minSoltutionRandom.state.timetable.get_batching_rules_for_task(task_id))

    if has_normal_br < 1:
        print(f"No Batching Rule found for {task_id}!")


for task_id in minSoltutionRandom.state.timetable.get_task_ids():
    has_normal_br = len(minSoltutionRandom.state.timetable.get_batching_rules_for_task(task_id))
    has_random_br = len(minSoltutionNormal.state.timetable.get_batching_rules_for_task(task_id))

    if has_normal_br < 1:
        print(f"No BR for {task_id}")
    if has_random_br < 1:
        print(f"No BR for {task_id}")


In [ ]:
# Analysis
sorted_tasks = sorted(
    storeNormal.current_evaluation.get_avg_cost_per_task().items(),
    key=lambda x: x[1],
    reverse=True,
)

avg_batching_waiting_time_per_task = storeNormal.current_evaluation.avg_batching_waiting_time_per_task
sorted_tasks = sorted(
    avg_batching_waiting_time_per_task.items(),
    key=lambda x: x[1],
    reverse=True,
)
print(sorted_tasks[0])

In [7]:
from o2.actions.batching_actions.modify_size_rule_by_cost_action import ModifySizeRuleByCostActionParamsType
from o2.models.rule_selector import RuleSelector
from o2.actions.batching_actions.modify_size_rule_by_cost_action import ModifySizeRuleByCostAction
from o2.util.logger import setup_logging
from o2.models.state import State
from o2.models.timetable import TimetableType


Settings.LOG_LEVEL = "IO"
Settings.LOG_FILE = "logs/playground.log"
Settings.SHOW_SIMULATION_ERRORS = True

setup_logging()

selector = RuleSelector(
    batching_rule_task_id="node_d45b421a-49fb-48c3-b8d9-fa77d9338e74", firing_rule_index=(0, 0)
)

state = minSoltutionNormal.state
duration_fn = storeNormal.constraints.get_duration_fn_for_task("node_d45b421a-49fb-48c3-b8d9-fa77d9338e74")

action = ModifySizeRuleByCostAction(
    ModifySizeRuleByCostActionParamsType(
        rule=selector,
        size_increment=1,
        duration_fn=duration_fn,
    )
)

state = State(
    bpmn_definition=state.bpmn_definition,
    timetable=TimetableType.from_json(state.timetable.to_json()),
    for_testing=state.for_testing,
)
new_state = action.apply(new_state)

print(new_state.timetable.to_json())

print(f"is valid? {new_state.is_valid()}")

new_state.evaluate()
# new_cycle_time = new_state.evaluate().total_cycle_time
# old_cycle_time = state.evaluate().total_cycle_time
# change = new_cycle_time - old_cycle_time


# print(f"New Cycle Time: {new_cycle_time:_.2f}")
# print(f"Old Cycle Time: {old_cycle_time:_.2f}")
# print(f"Change: {change:_.2f}")





2025-03-12 01:41:53,896 [DEBUG]: Logging to file: logs/playground.log


{"resource_profiles": [{"id": "10124.0_profile", "name": "10124.0_profile", "resource_list": [{"id": "10124.0", "name": "10124.0", "cost_per_hour": 20, "amount": 1, "calendar": "Undifferentiated_calendar", "assigned_tasks": ["node_9e98e80b-93db-44dc-bb5e-4bd783db0a6a", "node_1a561183-d10b-44f4-be71-e48ca4f32beb", "node_ffabd43c-3cfb-46c6-bf58-dc80aeed44c2"]}]}, {"id": "10125.0_profile", "name": "10125.0_profile", "resource_list": [{"id": "10125.0", "name": "10125.0", "cost_per_hour": 20, "amount": 1, "calendar": "Undifferentiated_calendar", "assigned_tasks": ["node_1a561183-d10b-44f4-be71-e48ca4f32beb"]}]}, {"id": "10138.0_profile", "name": "10138.0_profile", "resource_list": [{"id": "10138.0", "name": "10138.0", "cost_per_hour": 20, "amount": 1, "calendar": "Undifferentiated_calendar", "assigned_tasks": ["node_1a561183-d10b-44f4-be71-e48ca4f32beb", "node_a695624e-41e1-4480-8bac-9f0a5e46eb71", "node_3bbfedc8-a468-41b1-bd8d-90c225f67f43", "node_9e98e80b-93db-44dc-bb5e-4bd783db0a6a", "no

2025-03-12 01:41:53,925 [DEBUG]:         >>> Error in simulation: 'task_id'
2025-03-12 01:41:53,928 [INFO]: Traceback (most recent call last):
  File "/Users/jannis/Dropbox/Uni/Master/BP-Optimization/optimos_v2/o2/simulation_runner.py", line 44, in run_simulation
    setup = state.to_sim_diff_setup()
  File "/Users/jannis/Dropbox/Uni/Master/BP-Optimization/optimos_v2/o2/models/state.py", line 62, in to_sim_diff_setup
    setup = SimDiffSetupFileless(
  File "/Users/jannis/Dropbox/Uni/Master/BP-Optimization/optimos_v2/o2/util/sim_diff_setup_fileless.py", line 87, in __init__
    ) = self.parse_json_sim_parameters_from_string(timetable.to_dict())
  File "/Users/jannis/Dropbox/Uni/Master/BP-Optimization/optimos_v2/o2/util/sim_diff_setup_fileless.py", line 147, in parse_json_sim_parameters_from_string
    BatchProcessingParser(json_data[BATCH_PROCESSING_SECTION]).parse()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/opti2/lib/python3.10/site-packages/prosimos/batch_processing_parser.p

Evaluation(hourly_rates={}, task_kpis={}, resource_kpis={}, avg_cycle_time_by_case=0, avg_waiting_time_by_case=0, avg_batching_waiting_time_by_case=0, avg_batching_waiting_time_per_task={}, total_batching_waiting_time_per_task={}, total_duration=0, sum_of_durations=0, sum_of_cycle_times=0, total_batching_waiting_time=0, total_batching_waiting_time_per_resource={}, total_cycle_time=0, total_processing_time=0, total_waiting_time=0, task_execution_count_by_resource={}, is_empty=True, task_execution_count_with_wt_or_it={}, task_execution_counts={}, task_enablement_weekdays={}, task_started_weekdays={}, resource_allocation_ratio_task={}, total_fixed_cost_by_task={}, avg_fixed_cost_per_case=0, batches_by_activity_with_idle={}, avg_batch_size_for_batch_enabled_tasks=0, avg_batch_size_per_task={}, avg_idle_wt_per_task_instance=0, avg_batch_processing_time_per_task_instance=0, resource_started_weekdays={}, tasks_by_number_of_duplicate_enablement_dates={})

In [ ]:
from o2.models.days import DAY
from o2.models.time_period import TimePeriod

p = TimePeriod.from_start_end(1, 2, DAY.MONDAY)



In [19]:
import pickle
from o2.util.logger import setup_logging
from o2.models.settings import Settings
from o2.store import Store
from o2.util.solution_dumper import SolutionDumper
Settings.LOG_LEVEL = "IO"
Settings.LOG_FILE = "logs/playground.log"
Settings.SHOW_SIMULATION_ERRORS = True
Settings.ARCHIVE_SOLUTIONS = True

setup_logging()

SolutionDumper(global_mode=True)

store_path = "o2_evaluation/redumped_stores/store_tabu_search_random_gov_mid.pkl"
store:Store = pickle.load(open(store_path, "rb"))

#print(store.base_solution.evaluation.task_execution_counts)
keys_join = set(store.base_solution.evaluation.task_execution_counts.keys()).union(set(store.current_pareto_front.solutions[1].evaluation.task_execution_counts.keys()))
for key in keys_join:
    print(f"{key}")
    current_eval = store.current_pareto_front.solutions[1].evaluation
    count_base = store.base_solution.evaluation.task_execution_counts.get(key, 0)
    count_current = current_eval.task_execution_counts.get(key, 0)
    if key not in current_eval.task_kpis:
        print(f"Key {key} not in current evaluation")
        execution_time_current = 0
        idle_time_current = 0
        waiting_time_current = 0
        total_cycle_time_current = 0
        percentage_total_cycle_time = "0"
    else:
        execution_time_current = current_eval.task_kpis[key].processing_time.avg
        idle_time_current = current_eval.task_kpis[key].idle_time.avg
        waiting_time_current = current_eval.task_kpis[key].waiting_time.avg
        total_cycle_time_current = current_eval.task_kpis[key].idle_cycle_time.total
    if key not in store.base_solution.evaluation.task_kpis:
        print(f"\t> Key {key} not in base solution")
        execution_time_base = 0
        idle_time_base = 0
        waiting_time_base = 0
        total_cycle_time_base = 0
        percentage_total_cycle_time = "0"
    else:
        execution_time_base = store.base_solution.evaluation.task_kpis[key].processing_time.avg
        idle_time_base = store.base_solution.evaluation.task_kpis[key].idle_time.avg
        waiting_time_base = store.base_solution.evaluation.task_kpis[key].waiting_time.avg
        total_cycle_time_base = store.base_solution.evaluation.task_kpis[key].idle_cycle_time.total
        percentage_total_cycle_time = total_cycle_time_current / total_cycle_time_base

    
    print(f"\t> Count: {count_base} - {count_current}")
    print(f"\t> Execution Time: {execution_time_base} - {execution_time_current}")
    print(f"\t> Idle Time: {idle_time_base} - {idle_time_current}")
    print(f"\t> Waiting Time: {waiting_time_base} - {waiting_time_current}")
    print(f"\t> Total Cycle Time: {total_cycle_time_base} - {total_cycle_time_current}")
    print(f"\t> Time Increase {(percentage_total_cycle_time*100):.2f}%")



2025-03-17 09:54:31,511 [DEBUG]: Logging to file: logs/playground.log
2025-03-17 09:54:31,514 [IO]: Using global folders to load states and evaluations.


node130
	> Count: 15 - 8
	> Execution Time: 658532.1649121944 - 772363.0006187207
	> Idle Time: 114960.0 - 142554.25
	> Waiting Time: 0.0 - 0.0
	> Total Cycle Time: 11602382.473682914 - 7319338.004949765
	> Time Increase 63.08%
node8
	> Count: 2 - 4
	> Execution Time: 26.516620176613834 - 18.25617148861306
	> Idle Time: 11.48184009152228 - 8.52500988886209
	> Waiting Time: 0.0 - 0.0
	> Total Cycle Time: 75.99692053627223 - 107.1247255099006
	> Time Increase 140.96%
node11
	> Key node11 not in base solution
	> Count: 0 - 1
	> Execution Time: 0 - 75.81513961777091
	> Idle Time: 0 - 0.0
	> Waiting Time: 0 - 0.0
	> Total Cycle Time: 0 - 75.81513961777091


ValueError: Unknown format code 'f' for object of type 'str'

In [25]:
import pickle
from o2.util.logger import setup_logging
from o2.models.settings import Settings
from o2.store import Store
from o2.util.solution_dumper import SolutionDumper
Settings.LOG_LEVEL = "IO"
Settings.LOG_FILE = "logs/playground.log"
Settings.SHOW_SIMULATION_ERRORS = True
Settings.ARCHIVE_SOLUTIONS = True

setup_logging()

SolutionDumper(global_mode=True)

store_path = "o2_evaluation/redumped_stores/store_proximal_policy_optimization_random_bpic2019_das_hard.pkl"
store:Store = pickle.load(open(store_path, "rb"))

len([s for s in store.solution_tree.solution_lookup.values() if s is not None and s.is_valid])

2025-03-17 11:19:20,666 [DEBUG]: Logging to file: logs/playground.log
2025-03-17 11:19:20,669 [IO]: Using global folders to load states and evaluations.


469